In [1]:
import os
import glob
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib.backends.backend_pdf import PdfPages

plt.rcParams['figure.dpi'] = 400

In [2]:
# ---------------- Common Functions ---------------- #

def bg_flag(alpha, nrg):
    """
    Apply background flag filtering based on alpha and nrg values.

    Parameters:
        alpha : numpy array
            The array of computed alpha values.
        nrg   : numpy array
            The energy values, typically computed as 0.04 * pi.

    Returns:
        good  : numpy array (bool)
            A Boolean mask where True indicates a "good" event.
    """
    a1 = 0.35
    a2 = 0.7
    nrg0 = 2.0
    b1 = (1 - a1) / (5.5 - nrg0)
    b2 = (0.95 - a2) / (8.0 - nrg0)

    bad1 = alpha > (a1 + b1 * (nrg - nrg0))
    bad2 = alpha > (a2 + b2 * (nrg - nrg0))

    bad = bad1 | bad2  # Elementwise logical OR.
    good = ~bad        # Logical NOT.
    return good

######################################################################################################################################################

def extract_common_data(d, pi_min, pi_max, bgflag=None):
    """
    Extract common arrays used by both the XY and PI/Alpha plots.

    Parameters:
        d       : Data structure (dict or structured array) containing:
                  'pi', 'detx', 'dety', 'trk_m2l', 'trk_m2t', 'detphi', 'time'.
        pi_min  : Minimum PI value to consider.
        pi_max  : Maximum PI value to consider.
        bgflag  : Optional flag; if provided (not None), triggers background filtering
                  using the user-defined bg_flag function.

    Returns:
        common  : Dictionary containing:
                  "ok"      - initial boolean mask from selection,
                  "pi"      - filtered pi array (further filtered by bg_flag if requested),
                  "nrg_pi"  - computed energy values (0.04 * pi),
                  "alpha"   - computed alpha array (filtered if applicable),
                  "phi"     - d["detphi"] for the initially selected events.
    """
    # Create initial selection mask.
    ok = ((d["pi"] != 0) &
          (d["pi"] > pi_min) &
          (d["pi"] <= pi_max) &
          (np.abs(d["detx"]) < 7) &
          (np.abs(d["dety"]) < 7))
    
    pi = d["pi"][ok]
    nrg_pi = 0.04 * pi
    tl = d["trk_m2l"][ok]
    tw = d["trk_m2t"][ok]
    alpha = (tl - tw) / (tl + tw)
    
    # If bgflag is provided, filter alpha and pi using the bg_flag function.
    if bgflag is not None:
        good = bg_flag(alpha, nrg_pi)
        alpha = alpha[good]
        pi = pi[good]
    
    # For phi, we simply take it from the initially selected events.
    phi = d["detphi"][ok]
    
    return {"ok": ok, "pi": pi, "nrg_pi": nrg_pi, "alpha": alpha, "phi": phi}

In [3]:
# ---------------- Plotting Functions ---------------- #

def step_plot(x, y, binwidth):
    """
    Given arrays of bin centers (x) and counts (y),
    build "stepped" x and y arrays for a step plot.

    Parameters:
        x       : 1D array of bin centers.
        y       : 1D array of counts for each bin.
        binwidth: Width of a single bin.

    Returns:
        xsteps, ysteps : Lists containing x and y values for a step plot.
    """
    xsteps = []
    ysteps = []
    for i in range(len(x)):
        xsteps.append(x[i] - binwidth/2)
        xsteps.append(x[i] + binwidth/2)
        ysteps.append(y[i])
        ysteps.append(y[i])
    return xsteps, ysteps

######################################################################################################################################################

def plot_xy_vs_t(d, pi_min, pi_max, alpha_min, title, bgflag=None):
    """
    Replicates the IDL procedure "plot_xy_vs_t" in Python.

    Parameters:
        d         : Data structure (dict or structured array) containing the fields:
                    'pi', 'detx', 'dety', 'trk_m2l', 'trk_m2t', 'detphi', 'time'.
        pi_min    : Minimum PI value to consider.
        pi_max    : Maximum PI value to consider.
        alpha_min : Threshold on alpha used for overlaying in time plots.
        title : Title for all generated plots.
        bgflag    : If provided (not None), calls the bg_flag function to further filter
                    alpha and pi.
                    
    Returns:
        figs      : A list of matplotlib Figure objects:
                    [fig_hist, fig_phi_alpha, fig_detx_time, fig_dety_time, fig_detxy].
    """
    common = extract_common_data(d, pi_min, pi_max, bgflag=bgflag)
    pi = common["pi"]
    nrg_pi = common["nrg_pi"]
    alpha = common["alpha"]
    phi = common["phi"]
    ok = common["ok"]
    
    # ---- Figure 1: Histogram of phi ----
    dphi = 0.001 * np.pi
    bins = np.arange(-np.pi, np.pi + dphi, dphi)
    phist, edges = np.histogram(phi, bins=bins)
    phival = (edges[:-1] + edges[1:]) / 2

    # Use your step_plot function with the bin centers, histogram, and bin width.
    xsteps, ysteps = step_plot(phival, phist, dphi)
    
    fig_hist, ax_hist = plt.subplots()
    ax_hist.plot(xsteps, ysteps, 'b')
    ax_hist.set_xlabel(r'$\phi$')
    ax_hist.set_title(rf'{title} - $\phi$ Dist.')
    
    # ---- Figure 2: Plot phi vs. alpha ----
    fig_phi_alpha, ax_phi_alpha = plt.subplots()
    ax_phi_alpha.scatter(phi, alpha, c='b', s=1)
    ax_phi_alpha.set_xlabel(r'$\phi$')
    ax_phi_alpha.set_ylabel(r'$\alpha$')
    ax_phi_alpha.set_title(rf'{title} - $\phi$ vs $\alpha$')
    
    # Determine indices where alpha > alpha_min.
    wb = (alpha > alpha_min)
    
    # Precompute time with offset (for the whole dataset, not filtered by bg_flag).
    time_all = d["time"] - np.min(d["time"])
    
    # ---- Figure 3: Time vs. DetX with overlay ----
    fig_detx_time, ax_detx_time = plt.subplots()
    ax_detx_time.scatter(time_all, d["detx"], c='b', s=3)
    ax_detx_time.set_xlim(0, 400)
    ax_detx_time.set_xlabel("Time (s)")
    ax_detx_time.set_ylabel("DetX")
    ax_detx_time.set_title(f'{title} - DetX vs Time')
    ax_detx_time.scatter(d["time"][ok][wb] - np.min(d["time"]), d["detx"][ok][wb], c='r', s=1, label=rf'$\alpha$ > {alpha_min}')
    ax_detx_time.legend()
    
    # ---- Figure 4: Time vs. DetY with overlay ----
    fig_dety_time, ax_dety_time = plt.subplots()
    ax_dety_time.scatter(time_all, d["dety"], c='b', s=3)
    ax_dety_time.set_xlim(0, 400)
    ax_dety_time.set_xlabel("Time (s)")
    ax_dety_time.set_ylabel("DetY")
    ax_dety_time.set_title(f'{title} - DetY vs Time')
    ax_dety_time.scatter(d["time"][ok][wb] - np.min(d["time"]), d["dety"][ok][wb], c='r', s=1, label=rf'$\alpha$ > {alpha_min}')
    ax_dety_time.legend()
    
    # ---- Figure 5: DetX vs. DetY (first 10,000 events) with overlay ----
    fig_detxy, ax_detxy = plt.subplots()
    ax_detxy.scatter(d["detx"][:10000], d["dety"][:10000], c='b', s=3)
    ax_detxy.set_xlabel("DetX")
    ax_detxy.set_ylabel("DetY")
    ax_detxy.set_title(f'{title} - DetX vs DetY')
    ax_detxy.scatter(d["detx"][ok][wb], d["dety"][ok][wb], c='r', s=1, label=rf'$\alpha$ > {alpha_min}')
    ax_detxy.legend()
    
    return [fig_hist, fig_phi_alpha, fig_detx_time, fig_dety_time, fig_detxy]

######################################################################################################################################################

def plot_pi_alpha(d, pi_min, pi_max, title, bgflag=None):
    """
    Replicates the IDL procedure "plot_pi_alpha" in Python.

    Parameters:
        d         : Data structure (dict or structured array) that includes:
                    'pi', 'detx', 'dety', 'trk_m2l', 'trk_m2t', etc.
        pi_min    : Minimum PI value to consider.
        pi_max    : Maximum PI value to consider.
        title : Title for the plots.
        bgflag    : If provided (not None), calls the bg_flag function to further filter
                    alpha and pi.
    
    Returns:
        figs      : A list of two Matplotlib Figure objects:
                    [fig_alpha_hist, fig_pi_hist].
    """
    common = extract_common_data(d, pi_min, pi_max, bgflag=bgflag)
    pi = common["pi"]
    nrg_pi = common["nrg_pi"]
    alpha = common["alpha"]
    
    # ---- Plot 1: Histogram of α distribution ----
    bins_alpha = np.arange(0, 1.0, 0.01)
    ahist, edges_alpha = np.histogram(alpha, bins=bins_alpha)
    # Calculate bin centers.
    aval_alpha = (edges_alpha[:-1] + edges_alpha[1:]) / 2
    # Create step plot arrays; binwidth here is 0.01.
    xsteps_alpha, ysteps_alpha = step_plot(aval_alpha, ahist, 0.01)
    fig_alpha_hist, ax_alpha_hist = plt.subplots()
    ax_alpha_hist.plot(xsteps_alpha, ysteps_alpha, 'b')
    ax_alpha_hist.set_xlabel(r"$\alpha$")
    ax_alpha_hist.set_ylabel("Count")
    ax_alpha_hist.set_title(rf'{title} - $\alpha$ Dist.')

    # ---- Plot 2: Histogram of PI distribution ----
    bins_pi = np.arange(0, 301, 1)
    phist, edges_pi = np.histogram(pi, bins=bins_pi)
    aval_pi = (edges_pi[:-1] + edges_pi[1:]) / 2
    # Here, the binwidth is 1.
    xsteps_pi, ysteps_pi = step_plot(aval_pi, phist, 1)
    fig_pi_hist, ax_pi_hist = plt.subplots()
    ax_pi_hist.plot(xsteps_pi, ysteps_pi, 'b')
    ax_pi_hist.set_xlabel('PI')
    ax_pi_hist.set_ylabel("Count")
    ax_pi_hist.set_title(f'{title} - PI Dist.')
    
    return [fig_alpha_hist, fig_pi_hist]

######################################################################################################################################################

In [4]:
# ---------------- Fitting Functions ---------------- #

def minimizer(func, p0, args=(), tol=1e-6):
    """
    A general minimizer using the Nelder-Mead method.
    
    Parameters:
        func : callable
            The function to minimize. It should return a scalar value.
        p0   : array-like
            The initial guess for the parameters.
        args : tuple
            Additional arguments passed to 'func'.
        tol  : float
            Tolerance for convergence.
    
    Returns:
        The optimized parameters (numpy array).
    """
    res = minimize(func, p0, args=args, method='Nelder-Mead', tol=tol)
    return res.x

######################################################################################################################################################

def polar_likelihood_1d(param, ci):
    """
    Compute the polar likelihood for the given parameter.
    
    Parameters:
        param : float
            The modulation parameter to evaluate.
        ci    : numpy array
            Array of cosine modulation components (e.g. cos(2*phase) values) for the events.
    
    Returns:
        likelihood : float
            The computed likelihood value.
    
    The likelihood is computed as:
        likelihood = -2 * sum( ln(1 + param * ci) )
    """
    # Ensure the argument of the logarithm is positive.
    if np.any(1 + param * ci <= 0):
        raise ValueError("Encountered non-positive values in log argument.")
    
    likelihood = -2 * np.sum(np.log(1 + param * ci))
    
    return likelihood

######################################################################################################################################################

def invert_matrix(matrix):
    """
    Invert a matrix and return inversion status and the inverse.
    
    Parameters:
        matrix : 2D numpy array.
    
    Returns:
        inv_stat : int
           0 if inversion successful,
           1 if inversion failed,
           2 if small pivot encountered (accuracy questioned).
        inv_matrix : 2D numpy array, the inverse of matrix.
    
    Replace or extend this function as needed.
    """
    try:
        inv_matrix = np.linalg.inv(matrix)
        inv_stat = 0
    except np.linalg.LinAlgError:
        inv_matrix = None
        inv_stat = 1
    return inv_stat, inv_matrix

######################################################################################################################################################

def polar_likelihood(param, evtq, evtu):
    """
    Compute the 1D polar likelihood.
    
    In this parameterization, param[0] is interpreted as q and
    param[1] as u. The likelihood is defined as:
    
        likelihood = -2 * sum( log(1 + q*evtq + u*evtu) )
    
    Parameters:
        param : array-like
            A two-element array-like object where param[0] is q and param[1] is u.
        evtq  : numpy array
            Array of measured q_i values.
        evtu  : numpy array
            Array of measured u_i values.
    
    Returns:
        float : The computed likelihood.
    
    Raises:
        ValueError: If any value in (1 + q*evtq + u*evtu) is non-positive.
    """
    q = param[0]
    u = param[1]
    argument = 1 + q * evtq + u * evtu
    if np.any(argument <= 0):
        raise ValueError("Non-positive argument encountered in log")
    likelihood = -2 * np.sum(np.log(argument))
    return likelihood

######################################################################################################################################################

def polar_evpa_likelihood(param, evtq, evtu):
    """
    Compute the polarization/EVPA likelihood.
    
    In this parameterization:
        q = param[0] * cos(2 * param[1] * dtor)
        u = param[0] * sin(2 * param[1] * dtor)
    where dtor = pi/180 is the conversion from degrees to radians.
    
    The likelihood is computed as:
    
        likelihood = -2 * sum( log(1 + q*evtq + u*evtu) )
    
    Parameters:
        param : array-like
            A two-element array-like object; param[0] is the polarization amplitude and
            param[1] is the EVPA in degrees.
        evtq  : numpy array
            Array of measured q_i values.
        evtu  : numpy array
            Array of measured u_i values.
    
    Returns:
        float : The computed likelihood.
    
    Raises:
        ValueError: If any value in (1 + q*evtq + u*evtu) is non-positive.
    """
    dtor = np.pi / 180.0
    # Compute q and u using the EVPA parameterization.
    q = param[0] * np.cos(2 * param[1] * dtor)
    u = param[0] * np.sin(2 * param[1] * dtor)
    argument = 1 + q * evtq + u * evtu
    if np.any(argument <= 0):
        raise ValueError("Non-positive argument encountered in log")
    likelihood = -2 * np.sum(np.log(argument))
    return likelihood

######################################################################################################################################################

def pderiv(func, x, i, dx):
    """
    Estimate the first derivative (partial derivative) of a function with respect
    to x[i] using the central difference method.
    
    Parameters:
        func : callable
            The function to differentiate. It should accept a NumPy array x.
        x    : numpy.ndarray
            The point at which to compute the derivative.
        i    : int
            The index of the element of x with respect to which the derivative is taken.
        dx   : float
            The step size for the i-th element.
    
    Returns:
        float : The estimated derivative (∂f/∂x[i]).
    """
    x0 = x.copy()
    x1 = x.copy()
    x0[i] -= 0.5 * dx
    x1[i] += 0.5 * dx
    f0 = func(x0)
    f1 = func(x1)
    return (f1 - f0) / dx

######################################################################################################################################################

def pderiv2(func, x, dx):
    """
    Estimate the Hessian (matrix of second derivatives) of a function using central differences.
    
    For parameters x = [x0, x1, ..., x_{n-1}], the element [i, j] of the Hessian is approximated by:
    
        H[i,j] = ( pderiv(func, x_i+, dx[j]) - pderiv(func, x_i-, dx[j]) ) / dx[i]
    
    where x_i+ and x_i- are copies of x with the i-th element increased or decreased by 0.5*dx[i].
    
    Parameters:
        func : callable
            The function to differentiate. It should accept a NumPy array x.
        x    : numpy.ndarray
            The point at which to compute the Hessian.
        dx   : numpy.ndarray
            An array of step sizes for each component in x.
    
    Returns:
        numpy.ndarray: A 2D array (npar x npar) containing the approximate second derivatives.
    """
    npar = len(x)
    pder2_mat = np.zeros((npar, npar))
    
    for i in range(npar):
        for j in range(i, npar):
            x0 = x.copy()
            x1 = x.copy()
            x0[i] -= 0.5 * dx[i]
            x1[i] += 0.5 * dx[i]
            pd0 = pderiv(func, x0, j, dx[j])
            pd1 = pderiv(func, x1, j, dx[j])
            pder2_mat[i, j] = (pd1 - pd0) / dx[i]
    
    # Fill in the lower triangle by symmetry.
    for i in range(1, npar):
        for j in range(i):
            pder2_mat[i, j] = pder2_mat[j, i]
    
    return pder2_mat
    
######################################################################################################################################################

def likelihood(qmu, umu):
    """
    Estimate Q and U (and thus polarization and EVPA) via a likelihood method.
    
    Parameters:
        qmu : numpy array
            Array of measured Q values (or proxy; see IDL code, here called evtq).
        umu : numpy array
            Array of measured U values (or proxy; here called evtu).
    
    Returns:
        A tuple:
          (qu_simple, qu_simple_err, qu, qu_err, poln, poln_err, evpa, evpa_err, 
           delta_like, mdp)
    
    Note: Several user-defined functions (amoeba, pderiv2, invert_matrix, polar_likelihood,
          polar_evpa_likelihood) are required.
    """
    # Use the measured Q and U values.
    evtq = qmu.copy()
    evtu = umu.copy()
    
    # Nominal simple estimates.
    qu_simple_err = np.empty(2, dtype=float)
    qu_simple = np.empty(2, dtype=float)
    qu_simple_err[0] = 1.0 / np.sqrt(np.sum(evtq**2))
    qu_simple_err[1] = 1.0 / np.sqrt(np.sum(evtu**2))
    qu_simple[0] = np.sum(evtq) * (qu_simple_err[0]**2)
    qu_simple[1] = np.sum(evtu) * (qu_simple_err[1]**2)
    
    parm_est = qu_simple
    err_est = qu_simple_err
    # Compute nominal likelihood.
    # Here we call polar_likelihood with parameter parm_est and use evtq as "ci".
    like0 = polar_likelihood(parm_est, evtq, evtu)
    tolerance = abs(0.01 / like0)
    
    # Optimize Q and U using the amoeba routine.
    qu = minimizer(polar_likelihood, parm_est, args=(evtq, evtu), tol=tolerance)
    
    # Estimate uncertainties via the Hessian.
    hessian = 0.5 * pderiv2(lambda x: polar_likelihood(x, evtq, evtu), qu, err_est)
    inv_stat, err_matrix = invert_matrix(hessian)
    if inv_stat == 1:
        print("Inversion failed")
    elif inv_stat == 2:
        print("Small pivot -- accuracy questioned")
    qu_err = np.empty(2, dtype=float)
    qu_err[0] = np.sqrt(abs(err_matrix[0,0]))
    qu_err[1] = np.sqrt(abs(err_matrix[1,1]))
    
    # Cross–correlation (not used further, but computed)
    cross_corr = err_matrix[0,1] / (qu_err[0] * qu_err[1])
    
    # Minimum detectable polarization (MDP)
    mdp = 4.29 / np.sqrt(np.sum(evtq**2 + evtu**2))
    
    # Now use the polarization/EVPA version.
    poln_est = np.sqrt(np.sum(qu**2))
    if poln_est == 0:
        poln_err_est = 0
    else:
        poln_err_est = np.sqrt(np.sum((qu * qu_err / poln_est)**2))
    # EVPA estimate in degrees:
    rad2deg = 180.0/ np.pi
    evpa_est = 0.5 * np.arctan2(qu[1], qu[0]) * rad2deg
    evpa_err_est = 0.5 * np.sqrt((qu[1]*qu_err[0])**2 + (qu[0]*qu_err[1])**2) / (poln_est**2) * rad2deg
    
    parm_est_evpa = np.array([poln_est, evpa_est])
    err_est_evpa = np.array([poln_err_est, evpa_err_est])
    like0_evpa = polar_evpa_likelihood(parm_est_evpa, evtq, evtu)
    tolerance_evpa = abs(0.01 / like0_evpa)
    best_param = minimizer(polar_evpa_likelihood, parm_est_evpa, args=(evtq, evtu), tol=tolerance_evpa)
    poln = best_param[0]
    evpa = best_param[1]
    
    # Compute a grid search to determine the minimum likelihood value around the optimum.
    # We'll search over a grid spanning ±0.5*poln_err in the first parameter
    # and ±0.5*evpa_err in the second parameter.
    grid_p0, grid_p1 = np.meshgrid(
        np.linspace(best_param[0] - 0.5 * poln_err_est, best_param[0] + 0.5 * poln_err_est, 20),
        np.linspace(best_param[1] - 0.5 * evpa_err_est, best_param[1] + 0.5 * evpa_err_est, 20)
    )
    grid_params = np.column_stack((grid_p0.ravel(), grid_p1.ravel()))
    grid_likelihoods = np.array([polar_evpa_likelihood(param, evtq, evtu) for param in grid_params])
    min_likelihood_grid = np.min(grid_likelihoods)
    delta_like = polar_evpa_likelihood(best_param, evtq, evtu) - min_likelihood_grid
    
    hessian_evpa = 0.5 * pderiv2(lambda x: polar_evpa_likelihood(x, evtq, evtu), best_param, err_est_evpa)
    inv_stat_evpa, err_matrix_evpa = invert_matrix(hessian_evpa)
    if inv_stat_evpa == 1:
        print("Inversion (EVPA) failed")
    elif inv_stat_evpa == 2:
        print("Small pivot in EVPA inversion -- accuracy questioned")
    poln_err = np.sqrt(abs(err_matrix_evpa[0,0]))
    evpa_err = np.sqrt(abs(err_matrix_evpa[1,1]))
    
    return qu_simple, qu_simple_err, qu, qu_err, poln, poln_err, evpa, evpa_err, delta_like, mdp

######################################################################################################################################################

def fit_mu_alpha(d, pi_min, pi_max, nalpha, title, bgflag=None):
    """
    Convert the IDL fit_mu_alpha routine into Python.
    
    Parameters:
        d         : Data structure (e.g., a dict or structured array) with fields:
                    'pi', 'detx', 'dety', 'trk_m2l', 'trk_m2t', 'detphi', 'time'.
        pi_min    : Minimum PI value.
        pi_max    : Maximum PI value.
        nalpha    : Number of alpha bins.
        title     : Title for plots.
        bgflag    : If not None, background filtering is applied.
    
    Returns:
        A tuple: (outputs, figs)
        
        Where 'outputs' is a dictionary containing:
          "mu_weight"        : Weighted modulation factor (float)
          "mu_noweight"      : Unweighted modulation factor (float)
          "mu_noweight_err"  : Error on the unweighted modulation (float)
          "alpha_bins"       : Average alpha value in each bin (1D array of length nalpha)
          "mu_bins"          : Modulation factor in each alpha bin (1D array)
          "mu_bins_err"      : Error on the modulation factor in each bin (1D array)
          "nevt_bins"        : Number of events in each alpha bin (1D array)
        
        'figs' is a list of Matplotlib figure objects created during the routine.
    """
    # Get common data (user-defined function).
    common = extract_common_data(d, pi_min, pi_max, bgflag=bgflag)
    pi = common["pi"]
    nrg_pi = common["nrg_pi"]
    alpha = common["alpha"]
    phi = common["phi"]
    ok = common["ok"]

    figs = []
    
    # --- Plot the alpha distribution ---
    bins_alpha = np.arange(0, 1.0, 0.01)
    ahist, edges_alpha = np.histogram(alpha, bins=bins_alpha)
    # Calculate bin centers.
    aval_alpha = (edges_alpha[:-1] + edges_alpha[1:]) / 2
    # Create step plot arrays; binwidth here is 0.01.
    xsteps_alpha, ysteps_alpha = step_plot(aval_alpha, ahist, 0.01)
    fig_alpha, ax_alpha = plt.subplots()
    ax_alpha.plot(xsteps_alpha, ysteps_alpha, 'b')
    ax_alpha.set_xlabel(r"$\alpha$")
    ax_alpha.set_ylabel("Count")
    ax_alpha.set_title(rf'{title} - $\alpha$ Dist.')
    figs.append(fig_alpha)

    # --- Compute unweighted modulation factor from φ ---
    dphi = 0.001 * np.pi
    bins_phi = np.arange(-np.pi, np.pi + dphi, dphi)
    phist, edges_phi = np.histogram(phi, bins=bins_phi)
    phival = (edges_phi[:-1] + edges_phi[1:]) / 2

    evtq = np.cos(2 * phi)
    evtu = np.sin(2 * phi)
    
    # Compute the unweighted / weighted modulation and related parameters via your 'likelihood' function.
    qu_simple, qu_simple_err, qu, qu_err, mu_noweight, mu_noweight_err, \
        evpa, evpa_err, delta_like, mdp = likelihood(evtq, evtu)
    
    # Model for the φ histogram:
    model = len(phi) * (1 + qu[0] * np.cos(2 * phival) + qu[1] * np.sin(2 * phival)) / 2000.0

    # Step plot for φ histogram:
    xsteps_phi, ysteps_phi = step_plot(phival, phist, dphi)
    fig_phi, ax_phi = plt.subplots()
    ax_phi.plot(xsteps_phi, ysteps_phi, 'b', label="Data")
    ax_phi.plot(phival, model, 'r', label="Model")
    ax_phi.set_xlabel(r'$\phi$')
    ax_phi.set_title(rf'{title} - $\phi$ Dist.')
    ax_phi.legend()
    figs.append(fig_phi)

    # --- Compute weighted modulation factor ---
    w = alpha ** 0.75
    weighthist = np.zeros(2000, dtype=float)
    for i in range(2000):
        # Indices where |phi - (phival[i] + dphi/2)| < dphi/2.
        wbin = np.where(np.abs(phi - (phival[i] + dphi/2)) < dphi/2)[0]
        weighthist[i] = np.sum(w[wbin]) if wbin.size > 0 else 0.0

    evtq_bin = np.cos(2 * phival)
    evtu_bin = np.sin(2 * phival)
    qhat_weight = 2 * np.sum(evtq_bin * weighthist) / np.sum(weighthist)
    uhat_weight = 2 * np.sum(evtu_bin * weighthist) / np.sum(weighthist)
    mu_weight = np.sqrt(qhat_weight**2 + uhat_weight**2)
    
    # --- Compute phase for each event (for alpha binning) ---
    phase = phi - 0.5 * np.arctan2(qu[1], qu[0])
    phase = np.where(phase < 0, phase + 2*np.pi, phase)
    phase = np.mod(phase, np.pi)
    
    # --- Divide into nalpha bins ---
    dalpha = 1.0 / float(nalpha)
    alpha_i = dalpha * (np.arange(nalpha) + 0.5)  # nominal bin centers in alpha
    mu_arr = np.zeros(nalpha, dtype=float)
    mu_err_arr = np.zeros(nalpha, dtype=float)
    alpha_bar = np.zeros(nalpha, dtype=float)
    nevt = np.zeros(nalpha, dtype=float)
    
    # Minimizer loop over alpha bins
    for i in range(nalpha):
        sel = np.where(np.abs(alpha - alpha_i[i]) < dalpha/2)[0]
        if sel.size > 100:
            nevt[i] = float(sel.size)
            alpha_bar[i] = np.mean(alpha[sel])   # average alpha in this bin
            ci = np.cos(2 * phase[sel])
            sum_ci2 = np.sum(ci**2)
            if sum_ci2 == 0:
                par_est = 0.0
            else:
                par_est = np.sum(ci) / sum_ci2
            err_est = 1.0 / np.sqrt(sum_ci2)

            # Wrap initial guess in a 1D array to avoid deprecation warnings.
            p0 = np.array([float(par_est)])
            like0 = float(polar_likelihood_1d(p0[0], ci))
            opt_par = minimizer(polar_likelihood_1d, p0, args=(ci,), tol=abs(0.01 / like0))
            mu_arr[i] = opt_par[0]
            mu_err_arr[i] = np.sqrt(1.0 / np.sum(ci**2 / ((1 + mu_arr[i] * ci)**2)))
            
            # For debugging / additional visualization: phase histogram in each bin
            pbin = 0.01 * np.pi
            p_bins = np.arange(0, np.pi + pbin, pbin)
            phist_phase, p_edges = np.histogram(phase[sel], bins=p_bins)
            pval = (p_edges[:-1] + p_edges[1:]) / 2
            
            plotlabel = f"{title} - {alpha_i[i]-dalpha/2:.1f} < $\\alpha$ < {alpha_i[i]+dalpha/2:.1f}"
            fig_phase, ax_phase = plt.subplots()
            norm_hist = phist_phase / np.sum(phist_phase) if np.sum(phist_phase) > 0 else phist_phase
            ax_phase.scatter(pval, norm_hist, c='b', s=10, label="Data")
            ax_phase.plot(pval, 0.01*(1 + mu_arr[i]*np.cos(2*pval)), c='r', label="Model")
            ax_phase.set_xlabel("Phase")
            ax_phase.set_title(plotlabel)
            ax_phase.legend()
            figs.append(fig_phase)
        # If less than 100 events in this alpha bin, we leave mu_arr[i] as zero.
    
    # Return dictionary and figure list:
    outputs = {
        # Weighted/unweighted factors for the entire PI range
        "mu_weight": mu_weight,
        "mu_noweight": mu_noweight,
        "mu_noweight_err": mu_noweight_err,
        # Binned alpha results
        "alpha_bins": alpha_bar,   # <== "alpha_bar" from IDL
        "mu_bins": mu_arr,        # <== "mu" from IDL
        "mu_bins_err": mu_err_arr,
        "nevt_bins": nevt
    }
    return outputs, figs

In [6]:
# Change to your working directory.
os.chdir('/Users/leodrake/Documents/MIT/IXPE/ground_calib')

# Aggregate all FITS files matching the pattern.
fits_files = glob.glob("cal*.fits")
fits_files.sort()  # Ensure a consistent order

energies = []
low_results = []
peak_results = []

xy_pi_mins = [0, 0, 0, 0, 0, 0, 0]
xy_pi_maxs = [70, 70, 70, 70, 70, 70, 30]
alpha_mins = [0.4, 0.6, 0.5, 0.5, 0.5, 0.8, 0.95]
hi_pi_mins = [25, 30, 35, 45, 60, 75, 110]
hi_pi_maxs = [70, 80, 90, 100, 120, 140, 190]

# Create a single PDF to hold all plots, one per page.
with PdfPages("python_plots.pdf") as pdf:
    for i, fname in enumerate(fits_files[:]):
        print(f'Processing {fname}...')
        
        # Open the FITS file (assuming the data is in the second HDU; adjust if needed)
        data = fits.getdata(fname, 1)
        
        # Extract the energy value from filename.
        # Assumes filename format like: cal4p51_....fits
        try:
            energy_str_raw = fname.split('_')[0][3:]
            energy = energy_str_raw.replace("p", ".")
            energies.append(float(energy))
        except Exception:
            energy = "Unknown"
            energies.append(np.nan)

        # --- XY vs Time Plot ---
        # Define parameters specific to the XY vs Time plot.
        xy_pi_min = xy_pi_mins[i]
        xy_pi_max = xy_pi_maxs[i]
        xy_alpha_min = alpha_mins[i]           # Threshold for alpha used in overlay plots (adjust as needed)
        xy_title = f"IXPE Ground Cal {energy} keV, PI {xy_pi_min}-{xy_pi_max}"
        # Call the function (it returns a list of figures)
        #figs_xy = plot_xy_vs_t(data, xy_pi_min, xy_pi_max, xy_alpha_min, xy_title, bgflag=None)
        #for fig in figs_xy:
        #    pdf.savefig(fig)
        #    plt.close(fig)

        # --- PI vs Alpha Plot ---
        # Define parameters specific to the PI vs Alpha plot.
        alpha_pi_min = 1               # Lower limit for PI (example)
        alpha_pi_max = 300             # Upper limit for PI (example)
        alpha_pi_title = f"IXPE Ground Cal {energy} keV, PI {alpha_pi_min}-{alpha_pi_max}"
        # Call the function (it returns a list of two figures)
        alpha_figs_pi = plot_pi_alpha(data, alpha_pi_min, alpha_pi_max, alpha_pi_title, bgflag=None)
        for fig in alpha_figs_pi:
            pdf.savefig(fig)
            plt.close(fig)

        # --- Mu Alpha Fit ---

        print('Beginning Fit...')
        
        # Define parameters for the Mu Alpha Fit.
        # Here we assume the PI range for fitting is the same as for the other plots.
        lo_mu_pi_min = 1
        lo_mu_pi_max = hi_pi_mins[i]
        nalpha_val = 5  # Number of alpha bins
        lo_fit_title = f"IXPE Ground Cal {energy} keV, PI {lo_mu_pi_min}-{lo_mu_pi_max}"
        
        # Call the fit_mu_alpha function.
        # It now returns a tuple: (results, figs_fit), where results is a dictionary of fit outputs,
        # and figs_fit is a list of figure objects.
        low_dict, lo_figs_fit = fit_mu_alpha(data, lo_mu_pi_min, lo_mu_pi_max, nalpha_val, lo_fit_title, bgflag=None)
        low_results.append(low_dict)
        
        # Save each figure from the fit to the PDF.
        for fig in lo_figs_fit:
            pdf.savefig(fig)
            plt.close(fig)

        hi_mu_pi_min = hi_pi_mins[i]
        hi_mu_pi_max = hi_pi_maxs[i]
        nalpha_val = 10  # Number of alpha bins
        hi_fit_title = f"IXPE Ground Cal {energy} keV, PI {hi_mu_pi_min}-{hi_mu_pi_max}"

        peak_dict, hi_figs_fit = fit_mu_alpha(data, hi_mu_pi_min, hi_mu_pi_max, nalpha_val, hi_fit_title, bgflag=None)
        peak_results.append(peak_dict)

        print('Fit Complete\n')
        
        # Save each figure from the fit to the PDF.
        for fig in hi_figs_fit:
            pdf.savefig(fig)
            plt.close(fig)

print("All plots have been saved to 'python_plots.pdf'.")

Processing cal2p01_ixpe20191204T200000.076_1203_002132_1118.lv1_recon010.fits...
Beginning Fit...
Fit Complete

Processing cal2p29_ixpe20191203T190000.029_1203_002107_1118.lv1_recon010.fits...
Beginning Fit...
Fit Complete

Processing cal2p70_ixpe20191202T210000.081_1203_002066_1118.lv1_recon010.fits...
Beginning Fit...
Fit Complete

Processing cal2p98_ixpe20191201T190000.369_1203_002044_1118.lv1_recon010.fits...
Beginning Fit...
Fit Complete

Processing cal3p69_ixpe20191205T200000.067_1203_002146_1118.lv1_recon010.fits...
Beginning Fit...
Fit Complete

Processing cal4p51_ixpe20191130T190000.181_1203_002022_1118.lv1_recon010.fits...
Beginning Fit...
Fit Complete

Processing cal6p40_ixpe20191126T190000.020_1203_001984_1118.lv1_recon010.fits...
Beginning Fit...
Fit Complete

All plots have been saved to 'python_plots.pdf'.


In [7]:
def plot_mu_vs_sqrtalpha(ax, sqrtalpha_ref, mudiff_ref,
                         sqrtalpha_other, mudiff_other,
                         ref_energy, other_energy, model_params,
                         ref_marker, comp_marker):
    """
    Plot (mu_diff vs. sqrt(alpha)) comparing a 6.4 keV reference to another energy,
    plus a Gaussian curve based on model_params = (centroid, width^2, norm).

    Parameters:
        ref_marker: The marker character for the reference data (e.g., 'D')
        comp_marker: The marker character for the comparison data (e.g., obtained from idl_markers)
    """
    centroid, width_sq, norm_ = model_params

    # Compute Gaussian model over a fixed range of sqrt(alpha) values.
    svals = np.linspace(0.2, 1.0, 100)
    gauss = norm_ * np.exp(-0.5 * (svals - centroid)**2 / width_sq)

    # Plot the reference data using the provided marker.
    ax.plot(sqrtalpha_ref, mudiff_ref, ref_marker, c='b',
            label=f"{ref_energy:.2f} keV (Ref)")
    # Plot the comparison energy data using its corresponding marker.
    ax.plot(sqrtalpha_other, mudiff_other, comp_marker, c='deeppink',
            label=f"{other_energy:.2f} keV")

    # Plot a horizontal reference line at 0.
    ax.axhline(0, color='k', ls='--')

    # Plot the Gaussian model curve.
    ax.plot(svals, gauss, 'mediumspringgreen')

    # Set axis labels, title, and limits.
    ax.set_xlabel(r"$\sqrt{\alpha}$")
    ax.set_ylabel(r"$\mu \;-\; \mathrm{Model}$")
    ax.set_title(f"Peak PI: {ref_energy:.2f} vs. {other_energy:.2f} keV")
    ax.set_ylim(-0.2, 0.2)
    ax.legend(fontsize='small')


def compute_model(eplot, aa=-0.28, bb=0.2, cc=0.21, dd=1./24.):
    """
    Compute the IDL-like model: 
      mu = (1 / [(-aa - bb*E)^-4 + (-cc - dd*E)^-4])^0.25,
    handling non-finite values.
    """
    term1 = (-aa - bb * eplot)**(-4)
    term2 = (-cc - dd * eplot)**(-4)
    with np.errstate(divide='ignore', invalid='ignore'):
        model = (1.0 / (term1 + term2))**0.25
        model[~np.isfinite(model)] = np.nan
    return model


def generate_summary_plots(energies, peak_results, low_results, filename="python_model_modf.pdf"):
    """
    Create a multipage PDF of summary plots:
      1) Energies vs. (peak, low, combined) mu plus model curves.
      2) Alpha vs. mu for peak fits.
      3) Alpha vs. mu for low fits.
      4) Difference (mu - linear model).
      5) mu_diff vs. sqrt(alpha) (Gaussian correction) comparing 6.4 keV vs. others.
      6) Final parameter vs. PI plots (if ncal >= 7).
    """
    # --- Preliminary Filtering ---
    valid_indices = [
        i for i, (p, l, e) in enumerate(zip(peak_results, low_results, energies))
        if (p is not None) and (l is not None) and (not np.isnan(e))
    ]
    if not valid_indices:
        print("Error: No valid results found after processing files. Cannot generate summary plots.")
        return

    energies = np.array([energies[i] for i in valid_indices])
    peak_results = [peak_results[i] for i in valid_indices]
    low_results  = [low_results[i] for i in valid_indices]
    if len(energies) == 0:
        print("Error: No valid data remain after filtering. Cannot generate summary plots.")
        return

    print(f"\nGenerating summary plots for {len(energies)} valid calibrations in '{filename}'...")

    # --- Precompute Event Counts and Weighted mu ---
    tot_hi = np.array([np.sum(res["nevt_bins"]) for res in peak_results])
    tot_lo = np.array([np.sum(res["nevt_bins"]) for res in low_results])
    tot_evt = tot_hi + tot_lo
    valid_evt_mask = (tot_evt > 0)

    mu_noweight_hi     = np.array([res["mu_noweight"] for res in peak_results])
    mu_noweight_hi_err = np.array([res["mu_noweight_err"] for res in peak_results])
    mu_noweight_lo     = np.array([res["mu_noweight"] for res in low_results])
    mu_noweight_lo_err = np.array([res["mu_noweight_err"] for res in low_results])

    mu_noweight_combined = np.zeros_like(mu_noweight_hi)
    mu_noweight_combined[valid_evt_mask] = (
        (tot_hi[valid_evt_mask] * mu_noweight_hi[valid_evt_mask] +
         tot_lo[valid_evt_mask] * mu_noweight_lo[valid_evt_mask])
        / tot_evt[valid_evt_mask]
    )

    # --- Model Curves: Sample at eplot = 2.0 to 8.0 keV ---
    eplot = 6 * np.arange(1000) * 0.001 + 2
    mu_model_orig   = compute_model(eplot, dd=1./24.)    # All PI (Di Marco+)
    mu_model_better = compute_model(eplot, dd=1./18.5)    # Peak Only

    # --- Write Plots to a Multi-Page PDF ---
    with PdfPages(filename) as pdf:

        #####################################
        # Section 1: Energies vs. mu Plots
        #####################################
        fig, ax = plt.subplots()
        ax.errorbar(energies, mu_noweight_hi, yerr=mu_noweight_hi_err, c='b',
                    fmt='o', markersize=6, capsize=3, label="PI peak")
        ax.errorbar(energies, mu_noweight_lo, yerr=mu_noweight_lo_err, c='darkorange',
                    fmt='s', markersize=6, capsize=3, label="Low PI")
        ax.plot(energies, mu_noweight_combined, 'd',  c='deeppink', label="All PI")
        ax.plot(eplot, mu_model_better, 'mediumspringgreen', label="Peak Only")
        ax.plot(eplot, mu_model_orig, 'k--', label="All PI (Di Marco+)")
        ax.set_xlabel("Energy (keV)")
        ax.set_ylabel(r"$\mu$")
        ax.set_title("IXPE Ground Cal")
        ax.set_xlim(min(1.5, energies.min() * 0.9), max(8.0, energies.max() * 1.1))
        ax.legend(loc='best')
        pdf.savefig(fig)
        plt.close(fig)

        #####################################
        # Section 2: Alpha vs. mu for Peak Fits
        #####################################
        alpha_hi, mu_hi, ok_alpha_hi = [], [], []
        threshold_factor = 0.005
        for i in range(len(energies)):
            alpha_bins = peak_results[i]["alpha_bins"]
            mu_bins = peak_results[i]["mu_bins"]
            nevt_bins = peak_results[i]["nevt_bins"]
            threshold = threshold_factor * tot_evt[i] if tot_evt[i] > 0 else 0
            mask = (nevt_bins > threshold)
            alpha_hi.append(alpha_bins)
            mu_hi.append(mu_bins)
            ok_alpha_hi.append(mask)

        fig, ax = plt.subplots()
        idl_markers = ['P', 'X', '1', '+', 'x', 's', 'D', '^', 'v']
        for i, en in enumerate(energies):
            idx = i % len(idl_markers)
            ax.plot(alpha_hi[i][ok_alpha_hi[i]],
                    mu_hi[i][ok_alpha_hi[i]],
                    marker=idl_markers[idx],
                    linestyle='none', label=f"{en:.2f} keV")
        aplot_line = np.linspace(0, 1, 100)
        mu_model_line = 0.05 + 0.8 * aplot_line
        ax.plot(aplot_line, mu_model_line, 'k-')
        ax.set_ylim(0, 1)
        ax.set_xlabel(r"$\alpha$")
        ax.set_ylabel(r"$\mu$")
        ax.set_title(r"Peak PI: $\mu$ vs. $\alpha$")
        ax.legend(fontsize='small', ncol=2)
        pdf.savefig(fig)
        plt.close(fig)

        #####################################
        # Section 3: Alpha vs. mu for Low Fits
        #####################################
        alpha_lo, mu_lo, ok_alpha_lo = [], [], []
        for i in range(len(energies)):
            alpha_bins = low_results[i]["alpha_bins"]
            mu_bins = low_results[i]["mu_bins"]
            nevt_bins = low_results[i]["nevt_bins"]
            threshold = threshold_factor * tot_evt[i] if tot_evt[i] > 0 else 0
            mask = (nevt_bins > threshold)
            alpha_lo.append(alpha_bins)
            mu_lo.append(mu_bins)
            ok_alpha_lo.append(mask)
        fig, ax = plt.subplots()
        for i, en in enumerate(energies):
            idx = i % len(idl_markers)
            ax.plot(alpha_lo[i][ok_alpha_lo[i]],
                    mu_lo[i][ok_alpha_lo[i]],
                    marker=idl_markers[idx],
                    linestyle='none', label=f"{en:.2f} keV")
        ax.plot(aplot_line, mu_model_line, 'k-')
        ax.set_ylim(0, 1)
        ax.set_xlabel(r"$\alpha$")
        ax.set_ylabel(r"$\mu$")
        ax.set_title(r"Low PI: $\mu$ vs. $\alpha$")
        ax.legend(fontsize='small', ncol=2)
        pdf.savefig(fig)
        plt.close(fig)

        #####################################
        # Section 4: Difference (mu - Linear Model)
        #####################################
        aa_model, bb_model = 0.05, 0.8
        mu_diff = []
        for i in range(len(energies)):
            diff = np.full_like(mu_hi[i], np.nan)
            mask_valid = ok_alpha_hi[i] & np.isfinite(alpha_hi[i]) & np.isfinite(mu_hi[i])
            diff[mask_valid] = mu_hi[i][mask_valid] - (aa_model + bb_model * alpha_hi[i][mask_valid])
            mu_diff.append(diff)
        fig, ax = plt.subplots()
        for i, en in enumerate(energies):
            idx = i % len(idl_markers)
            mask_valid = ok_alpha_hi[i] & np.isfinite(mu_diff[i])
            ax.plot(alpha_hi[i][mask_valid], mu_diff[i][mask_valid],
                    marker=idl_markers[idx], linestyle='none', label=f"{en:.2f} keV")
        ax.axhline(0, color='k', ls='--')
        ax.set_xlabel(r"$\alpha$")
        ax.set_ylabel(r"$\mu$ - Model")
        ax.set_title("Peak PI: Model Improvement")
        ax.set_ylim(-0.2, 0.2)
        ax.legend(fontsize='small', ncol=2)
        pdf.savefig(fig)
        plt.close(fig)

        #####################################
        # Section 5: Compute sqrt(alpha) for mu - model plots
        #####################################
        salpha_hi = []
        for i in range(len(energies)):
            vals = np.full_like(alpha_hi[i], np.nan)
            mask = ok_alpha_hi[i] & (alpha_hi[i] >= 0) & np.isfinite(alpha_hi[i])
            vals[mask] = np.sqrt(alpha_hi[i][mask])
            salpha_hi.append(vals)

        #####################################
        # Section 6: Define Hard-Coded Gaussian Parameters for Other Energies
        #####################################
        ncal = len(energies)
        if ncal >= 7:
            centroid_params = np.array([0.6, 0.58, 0.56, 0.55, 0.54, 0.53])
            width_sq_params = np.array([0.03, 0.03, 0.02, 0.02, 0.02, 0.05])
            norm_params = np.array([0.07, 0.12, 0.15, 0.17, 0.07, -0.035])
        else:
            print(f"Warning: Only {ncal} valid energies. Gaussian parameters might be misaligned.")
            centroid_params = np.array([])
            width_sq_params = np.array([])
            norm_params = np.array([])

        param_for_energy = {
            2.01: (0.53, 0.05, -0.035),
            2.29: (0.54, 0.02,  0.07),
            2.70: (0.55, 0.02,  0.17),
            2.98: (0.56, 0.02,  0.15),
            3.69: (0.58, 0.03,  0.12),
            4.51: (0.60, 0.03,  0.07)
        }

        #####################################
        # Section 7: Identify 6.4 keV Reference and Prepare Data
        #####################################
        try:
            idx_ref = list(energies).index(6.4)
        except ValueError:
            idx_ref = np.argmin(np.abs(energies - 6.4))
            print(f"Note: exact 6.4 keV not found. Using energies[{idx_ref}]={energies[idx_ref]:.2f} keV as reference.")
        salpha_ref = salpha_hi[idx_ref]
        mudiff_ref = mu_diff[idx_ref]
        mask_ref = ok_alpha_hi[idx_ref] & np.isfinite(salpha_ref) & np.isfinite(mudiff_ref)
        salpha_ref = salpha_ref[mask_ref]
        mudiff_ref = mudiff_ref[mask_ref]

        #####################################
        # Section 8: Loop Over Other Energies and Plot mu_diff vs. sqrt(alpha)
        #####################################
        for i, en_i in enumerate(energies):
            if i == idx_ref:
                continue
            if en_i not in param_for_energy:
                continue
            salpha_i = salpha_hi[i]
            mudiff_i = mu_diff[i]
            mask_i = ok_alpha_hi[i] & np.isfinite(salpha_i) & np.isfinite(mudiff_i)
            salpha_i = salpha_i[mask_i]
            mudiff_i = mudiff_i[mask_i]

            fig, ax = plt.subplots()
            plot_mu_vs_sqrtalpha(
                ax,
                sqrtalpha_ref=salpha_ref, mudiff_ref=mudiff_ref,
                sqrtalpha_other=salpha_i, mudiff_other=mudiff_i,
                ref_energy=energies[idx_ref], other_energy=en_i,
                model_params=param_for_energy[en_i],
                ref_marker=idl_markers[idx_ref % len(idl_markers)],    # use marker based on reference index
                comp_marker=idl_markers[i % len(idl_markers)]           # use marker based on current energy index
            )
            pdf.savefig(fig)
            plt.close(fig)

        #####################################
        # Section 9: Final Parameter vs. PI Plots (if exactly 7 energies)
        #####################################
        if ncal >= 7:
            pi_hi = np.array([150.0, 107.5, 90.0, 72.5, 62.5, 55.0, 47.5])
            pivalues = pi_hi * 1.05

            centroid = np.array([0.65] + centroid_params.tolist())
            norm = np.array([0.0] + norm_params.tolist())
            width_sq = np.array([0.03] + width_sq_params.tolist())

            if any(len(arr) != 7 for arr in [pivalues, centroid, norm, width_sq]):
                print("Warning: Parameter array mismatch for plots 11-13. Truncating or skipping.")
            else:
                piplot = np.arange(150, dtype=float) + 40

                # (11) Centroid vs. PI
                fig, ax = plt.subplots()
                ax.plot(pivalues, centroid, 'D', c='b', linestyle='none')
                if len(pivalues) > 1:
                    coeff = np.polyfit(pivalues, centroid, 1)
                    ax.plot(piplot, np.polyval(coeff, piplot), 'mediumspringgreen')
                ax.set_xlabel("PI (adjusted)")
                ax.set_ylabel(r"Centroid of $\Delta \mu$")
                ax.set_title("Centroid Parameter vs PI")
                pdf.savefig(fig)
                plt.close(fig)

                # (12) Norm vs. PI (piecewise)
                fig, ax = plt.subplots()
                ax.plot(pivalues, norm, 'D', c='b', linestyle='none')
                if len(pivalues) >= 7:
                    idx1 = [4, 5, 6]
                    idx2 = [1, 2, 3, 4]
                    coeff1 = np.polyfit(pivalues[idx1], norm[idx1], 1)
                    coeff2 = np.polyfit(pivalues[idx2], norm[idx2], 1)
                    norm_model = np.full_like(piplot, np.nan, dtype=float)
                    low_mask = (piplot < 66)
                    norm_model[low_mask] = np.polyval(coeff1, piplot[low_mask])
                    mid_mask = ~low_mask
                    norm_model[mid_mask] = np.polyval(coeff2, piplot[mid_mask])
                    ax.plot(piplot, norm_model, 'mediumspringgreen')
                ax.set_xlabel("PI (adjusted)")
                ax.set_ylabel(r"Norm of $\Delta \mu$")
                ax.set_title("Norm Parameter vs PI")
                pdf.savefig(fig)
                plt.close(fig)

                # (13) Width^2 vs. PI (IDL-style exact reproduction)
                fig, ax = plt.subplots()
                piplot = np.arange(150, dtype=float) + 40
                sigma2_extra = np.ones_like(piplot, dtype=float)
                low_mask = (piplot >= 55) & (piplot < 85)
                sigma2_extra[low_mask] = 0.02
                mid_mask = (piplot >= 85) & (piplot < 150)
                sigma2_extra[mid_mask] = 0.03
                ax.plot(pivalues, width_sq, 'D', c='b', linestyle='none')
                ax.plot(piplot, sigma2_extra, 'mediumspringgreen')
                ax.set_xlim(50, 150)
                ax.set_ylim(0.01, 0.04)
                ax.set_xlabel("PI (adjusted)")
                ax.set_ylabel(r"$\sigma^2$ of $\Delta \mu$")
                ax.set_title("Width Parameter ($\\sigma^2$) vs PI (IDL-style)")
                pdf.savefig(fig)
                plt.close(fig)
        else:
            print("Skipping plots 11-13: need at least 7 valid energies.")

    print(f"Summary plots saved to '{filename}'.")

#####################################
 # Exectuion
#####################################

generate_summary_plots(energies, peak_results, low_results)


Generating summary plots for 7 valid calibrations in 'python_model_modf.pdf'...
Summary plots saved to 'python_model_modf.pdf'.
